In [3]:
# This script contains the generator code for producing wireless network layouts and channel losses
# for the work "Spatial Deep Learning for Wireless Scheduling",
# available at https://ieeexplore.ieee.org/document/8664604.

# For any reproduction, further research or development, please kindly cite our JSAC journal paper:
# @Article{spatial_learn,
#    author = "W. Cui and K. Shen and W. Yu",
#    title = "Spatial Deep Learning for Wireless Scheduling",
#    journal = "{\it IEEE J. Sel. Areas Commun.}",
#    year = 2019,
#    volume = 37,
#    issue = 6,
#    pages = "1248-1261",
#    month = "June"

import numpy as np
import math
import time
import scipy.io as sio
import matplotlib.pyplot as plt

### Detailed Mathematical Model of `get_mu` Function

- $P_{\text{max}} $: Maximum transmit power.
- $\mathbf{H}_{kk} $: Channel matrix for the $k^{th} $ user.
- $\mathbf{\mu} $: Precoding weight vector,
- $\sigma^2 $ :noise variance,

#### Objective:
The objective is to find the optimal precoding weight vector $\mathbf{\mu} $ that maximizes the signal-to-interference-plus-noise ratio (SINR) while satisfying the power constraint $P_{\text{max}} $.

#### SINR Formulation:
The SINR for the $k^{th} $ user is given by: <br><br>
$ \boxed{ \text{SINR}_k = \frac{|(\mathbf{H}_k^H \mathbf{\mu})|^2}{\sum_{i \neq k} |(\mathbf{H}_k^H \mathbf{\mu})|^2 + \sigma^2} } $ <br><br>

#### Power Constraint:
The total power constraint is expressed as: <br><br>
$ \boxed{ \sum_{i=1}^{N} |\mu_i|^2 \leq P_{\text{max}} } $ <br><br>

#### Optimization Problem:
The problem can be formulated as:<br><br>
$ \boxed{ \underset{\mathbf{\mu}}{\text{maximize}} \quad \text{SINR}_k } $ <br><br>
$ \boxed{ \text{subject to} \quad \sum_{i=1}^{N} |\mu_i|^2 \leq P_{\text{max}} } $ <br><br>

#### Solution Approach:
Various optimization techniques can be employed to solve this problem, such as Lagrange multipliers or convex optimization methods.

#### Detailed Implementation:
The `get_mu` function implements an iterative approach to find the optimal $\mathbf{\mu} $. It utilizes eigenvalue decomposition and binary search to satisfy the power constraint.

In [ ]:
def get_mu(Pmax,Hkk,btmp,wf):
    Lmu = 0
    N = Hkk.shape[1]

    I = np.eye(N)
    Hkk_H = (Hkk.conj()).T
    
    if(np.linalg.matrix_rank(btmp) == N 
        and np.linalg.norm(np.matmul(np.linalg.inv(btmp),Hkk_H ) * wf) < np.sqrt(Pmax)):
        return np.squeeze(np.matmul(np.linalg.inv(btmp),Hkk_H ) * wf)

    Lambda, D = np.linalg.eig(btmp)
    Lambda = np.diag(Lambda)
    HUW = Hkk_H*wf
    Phitmp = np.matmul(HUW,(HUW.conj()).T)
    DH = (D.conj()).T

    Phi = np.matmul(np.matmul(DH,Phitmp),D)
    Phimm = np.real(np.diag(Phi))
    Lambdamm = np.real(np.diag(Lambda))

    
    Rmu = 1
    Pcomp = np.sum(Phimm/(Lambdamm + Rmu)**2)
    while(Pcomp > Pmax):
        Rmu = Rmu*2
        Lmu = Rmu
        Pcomp = np.sum(Phimm/(Lambdamm + Rmu)**2)
    while(Rmu-Lmu > 1e-4):
        midmu = (Rmu + Lmu)/2
        Pcomp = np.sum(Phimm/(Lambdamm + midmu)**2)
        if(Pcomp < Pmax ):
            Rmu = midmu
        else: 
            Lmu = midmu
    ans = np.squeeze(np.matmul(np.linalg.inv(btmp + Rmu*I),Hkk_H ) * wf)

    return ans

The code implements the Weighted Minimum Mean Squared Error (WMMSE) algorithm for precoding in a multi-user MIMO system. This algorithm aims to optimize the precoding vectors `b` to minimize the weighted sum of mean squared errors (MSE) subject to power constraints.

To explain the concept in a mathematical equation, let's define the objective function of the WMMSE algorithm:<br><br>
$ \boxed{\min_{b} \sum_{k=1}^{K} w_k \cdot \text{MSE}_k} $ <br><br>

where:
- $ K $  is the number of users (or transmit antennas).
- `b` is the precoding matrix (or vector) to be optimized.
- $ w_{k} $  are the weights associated with each user, typically inversely proportional to the noise variance at each user.
- $ \text{MSE}_{k} $  is the mean squared error for the $ k $ th user, defined as: <br><br> 
  $ \text{MSE}_k = \frac{1}{N} \sum_{n=1}^{N} \left| y_k(n) - \sum_{i=1}^{K} h_{k,i}(n) \cdot b_i \right|^2 $ <br><br>
  where:
  - $ N $  is the number of symbols transmitted.
  - $ y_{k}(n) $  is the received signal at the $ k $ th user.
  - $ h_{k,i}(n) $  is the channel coefficient between the $ i $ th transmit antenna and the $ k $ th user for symbol $ n $ .
  - $ b_i $  is the $ i $ th element of the precoding vector.

The algorithm iteratively updates the precoding vectors `b` to minimize this objective function while satisfying power constraints. The `get_mu` function appears to be used to compute a Lagrange multiplier or step size for the optimization.

The mathematical equation provided here explains the concept behind the WMMSE algorithm implemented in the code.


In [4]:
def np_WMMSE_vector(b_int, H, Pmax, var_noise):
    # fix transpose and conjudgate
    K = b_int.shape[0]
    N = b_int.shape[1]
    vnew = 0
    b = b_int
    f = np.zeros(K,dtype=complex)
    w = np.zeros(K,dtype=complex)

    mask = np.eye(K)

    btmp = np.reshape(b, (K,1,N))
    rx_power = np.multiply(H, b)
    rx_power = np.sum(rx_power,axis=-1)
    valid_rx_power = np.sum(np.multiply(rx_power, mask), axis=0)
    interference_rx = np.square(np.abs(rx_power))
    interference = np.sum(interference_rx, axis=1) + var_noise
    f = np.divide(valid_rx_power, interference)
    w = 1/(1 - valid_rx_power*f.conj())
    vnew = np.sum(np.log2(np.abs(w)))
    
    for iter in range(100):
        vold = vnew
        H_H = np.expand_dims(H.conj(),axis=-1)
        H_tmp = np.expand_dims(H,axis=-2)
        HH = np.matmul(H_H,H_tmp)

        UWU = np.reshape(w * (f.conj()).T * f,(K,1,1,1))
        btmp = np.sum(HH * UWU, axis=0)
        for ii in range(K):
            Hkk = np.expand_dims(H[ii,ii,:],axis=0)
            b[ii,:] = get_mu(Pmax,Hkk,btmp[ii,:,:],w[ii] * f[ii])

        btmp = np.reshape(b, (K,1,N))
        rx_power = np.multiply(H, b)
        rx_power = np.sum(rx_power,axis=-1)
        valid_rx_power = np.sum(np.multiply(rx_power, mask), axis=0)
        interference_rx = np.square(np.abs(rx_power))
        interference = np.sum(interference_rx, axis=1) + var_noise
        f = np.divide(valid_rx_power, interference)
        w = 1/(1 - valid_rx_power*f.conj())
        vnew = np.sum(np.log2(np.abs(w)))
        if abs(vnew - vold) <= 1e-3:
           break
    return b

The `IC_sum_rate` function calculates the sum rate of an interference channel (IC) based on the given channel matrix `H`, power allocation `p`, and noise variance `var_noise`.

### Concept Explanation:

1. **Received Signal Power (rx_power)**:
   - The received signal power for each user ` k `  is computed by element-wise multiplication of the channel matrix `H` and the power allocation matrix `p`: <br><br>
     $\text{rx\_power}_{k} = \sum_{i=1}^{K} \text{H}_{ki} \cdot \text{p}_{ki}$ <br><br>
   - This operation is performed over the transmit antennas and the result is squared to obtain the actual power:<br><br>
     $  \text{rx\_power}_{k} = \left( \sum_{i=1}^{K} H_{ki} \cdot p_{ki} \right)^2 $ <br><br>
   - These calculations are performed for all users ` k `.
2. **Interference Calculation (`interference`)**:
- The interference for each user `k` is calculated as the total received power minus the power received from its transmission (using a mask): <br><br>
$  \text{interference}_{k} = \left( \sum_{i=1}^{K} \text{rx\_power}_{i} \right) - \text{rx\_power}_{trans} + \sigma $<br><br>

3. **Achievable Rate (`rate`)**:
- The achievable rate for each user `k` is computed using the Shannon formula:<br><br>
$  \text{rate}_{k} = \log_2(1 + \text{SNR}_{k}) $ <br><br>
- where $ \text{SNR}_{k}$ is the signal-to-interference-plus-noise ratio for user ` k `:<br><br>
$  \text{SNR}_{k} = \dfrac{\text{rx\_power}_{k}}{\text{interference}_{k}} $

4. **Sum Rate Calculation (`sum_rate`)**:
- Finally, the sum rate is computed as the mean of the total rate achieved by each user:<br><br>
$  \text{sum\_rate} = \dfrac{1}{K} \sum_{k=1}^{K} \text{rate}_{k} $ <br><br>

The function returns the sum rate, which represents the total data rate achieved over all users in the interference channel.


In [6]:
def IC_sum_rate(H,p, var_noise):
    # H1 K*K*N
    # p1 K*N
    K = H.shape[1]
    N = H.shape[-1]
    p = p.reshape((-1,K,1,N))
    rx_power = np.multiply(H, p)
    rx_power = np.sum(rx_power,axis=-1)
    rx_power = np.square(np.abs(rx_power))
    mask = np.eye(K)
    valid_rx_power = np.sum(np.multiply(rx_power, mask), axis=1)
    interference = np.sum(np.multiply(rx_power, 1 - mask), axis=1) + var_noise
    rate = np.log2(1 + np.divide(valid_rx_power, interference))
    sum_rate = np.mean(np.sum(rate, axis=1))
    return sum_rate